In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import constants
import utils

Scrape a list of all players from fantasypros

In [2]:
PLAYERS_URL = 'https://www.fantasypros.com/mlb/rankings/ros-overall.php'
HEADERS = ['Rank', 'Player', 'Best', 'Worst', 'Avg', 'Std Dev', 'ADP', 'vs. ADP', 'Notes']

In [11]:
site_data = requests.get(PLAYERS_URL).text
soup = BeautifulSoup(site_data)
table = soup.find('table')

In [8]:
players = []
for row in table.tbody.find_all('tr'):
    player = []
    for data in row.find_all('td'):
        player.append(data.text.strip())
    players.append(player)

In [10]:
df = pd.DataFrame(players, columns=HEADERS)
# Remove unused column
df.drop(columns=['Notes'], inplace=True)
utils.save_dataframe(df, constants.FANTASYPROS_PLAYERS_FILENAME)